In [1]:
from transformers import (
    AutoConfig,
    AutoModelForSeq2SeqLM,
    AutoTokenizer,
    DataCollatorForSeq2Seq,
    HfArgumentParser,
    MBartTokenizer,
    default_data_collator,
    AutoModelWithLMHead,
    set_seed
)
model_name = "./models/First/"
model = AutoModelWithLMHead.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

if tokenizer.encode("<extra_id_0> <extra_id_1>") != [32099, 32098, 32097, 1]:
    # For non-t5 tokenizer
    tokenizer.add_special_tokens(
        {"additional_special_tokens": ["<Temp_S>", "<Temp_E>", "<Relation_S>", "<Relation_E>", \
            "<ORG>", "<VEH>", "<WEA>", "<LOC>","<FAC>","<End>" ,"<PER>","<GPE>"]})


/home/pesante/anaconda3/envs/text2event/lib/python3.8/site-packages/transformers/models/auto/modeling_auto.py:1006: FutureWarning: The class `AutoModelWithLMHead` is deprecated and will be removed in a future version. Please use `AutoModelForCausalLM` for causal language models, `AutoModelForMaskedLM` for masked language models and `AutoModelForSeq2SeqLM` for encoder-decoder models.
  warnings.warn(


In [19]:
b = tokenizer.convert_tokens_to_ids(["<Temp_S>", "<Temp_E>", "<Relation_S>", "<Relation_E>"])
print(b)

[32100, 32110, 32101, 32102]


In [24]:
tokenizer.add_special_tokens(
        {"additional_special_tokens": ["<Temp_S>", "<Temp_E>", "<Relation_S>", "<Relation_E>", \
            "<ORG>", "<VEH>", "<WEA>", "<LOC>","<FAC>","<End>" ,"<PER>","<GPE>"]})
b = tokenizer.convert_tokens_to_ids(["<Temp_S>", "<Temp_E>", "<Relation_S>", "<Relation_E>"])
print(b)
print(tokenizer.convert_tokens_to_ids(["<Temp_S>"])[0])
if tokenizer.convert_tokens_to_ids(["<Temp_S>"])[0] == 32100:
    print("kakd")

[32100, 32110, 32101, 32102]
32100
kakd


In [23]:
def add_space(text):
    """
    add space between special token
    :param text:
    :return:
    """
    new_text_list = list()
    for item in zip(split_bracket.findall(text), split_bracket.split(text)[1:]):
        new_text_list += item
    return ' '.join(new_text_list)

# Read Data

In [18]:
from datasets import load_dataset, load_metric
data_files = {}
data_files["train"] = './data/new_text2tree/one_ie_ace2005_subtype/train.json'
extension = 'json'
datasets = load_dataset(extension, data_files=data_files)

Using custom data configuration default-521fb87c8c936007


  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

Dataset json downloaded and prepared to /home/pesante/.cache/huggingface/datasets/json/default-521fb87c8c936007/0.0.0/c2d554c3377ea79c7664b93dc65d0803b45e3279000f993c7bfd18937fd7f426. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

In [19]:
column_names = datasets["train"].column_names
print(column_names)

['text', 'relation']


In [11]:
train_dataset = datasets["train"]
text_column = "text"
summary_column = "event"
inputs = train_dataset[text_column]
targets = train_dataset[summary_column]
padding = "max_length"
model_inputs = tokenizer(inputs, max_length= 256, padding=padding, truncation=True)

In [17]:
print(type(model_inputs))

<class 'transformers.tokenization_utils_base.BatchEncoding'>


# New data format

In [2]:
import os
import json
from collections import Counter, defaultdict
from data_convert.format.text2tree import Text2Tree
from data_convert.task_format.event_extraction import Event, DyIEPP
from data_convert.utils import read_file, check_output, data_counter_to_table, get_schema, output_schema
from nltk.corpus import stopwords

target_class=Text2Tree
type_format='subtype'

In [5]:
import os
in_filename = "data/raw_data/ace05-EN/train.oneie.json"
output_filename = "data/new_text2tree/ace2005_event/dev"
if not os.path.exists(output_filename):
        os.makedirs(output_filename)
event_output = open(output_filename + '.json', 'w')

count = 0
number = 0
for line in read_file(in_filename):
    document = Event(json.loads(line.strip()))
    if len(document.entities) > 3:
        break
    for sentence in document.generate_relations():
        number += 1
        if(len(sentence['relations']) == 0):
            count += 1
print(count)
print(number)

  0%|          | 6/17172 [00:00<00:00, 37560.93it/s]

6
6


In [12]:
import os
in_filename = "data/raw_data/ace05-EN/train.oneie.json"
output_filename = "data/new_text2tree/ace2005_event/dev"
if not os.path.exists(output_filename):
        os.makedirs(output_filename)
event_output = open(output_filename + '.json', 'w')

count = 0
number = 0
for line in read_file(in_filename):
    document = Event(json.loads(line.strip()))
    sentences = document.generate_relations()
    for sentence in sentences:
        if(len(sentence['relations']) > 0):
            print(sentence['relations'])
            break
    break

  0%|          | 0/17172 [00:00<?, ?it/s]


In [9]:
for sentence in document.generate_relations():
    print(sentence['relations'])
    print(" ")

[]
 


In [16]:
import os
in_filename = "data/datasets/conll04/conll04_train.json"
count = 0
number = 0
entity_set = set()
relation_type_set = set()
for line in read_file(in_filename):
    for line_ in json.loads(line.strip()):
        for entity in line_['entities']:
            entity_set.add(entity['type'])
        for relation in line_['relations']:
            relation_type_set.add(relation['type'])


100%|██████████| 1/1 [00:00<00:00, 127.34it/s]


In [17]:
print(entity_set)
print(relation_type_set)
conll04_train_dev
/data/datasets/conll04

{'Org', 'Peop', 'Other', 'Loc'}
{'Live_In', 'Kill', 'Located_In', 'OrgBased_In', 'Work_For'}


In [7]:
for relations_in_sentence, sentence_start, sentence, entity in zip(document.relations, document.sentence_start, document.sentences, document.ner):
    print(sentence)
    print(relations_in_sentence)
    print(entity)
    print(" ")

['English', 'is', 'shown', 'to', 'be', 'trans-context-free', 'on', 'the', 'basis', 'of', 'coordinations', 'of', 'the', 'respectively', 'type', 'that', 'involve', 'strictly', 'syntactic', 'cross-serial', 'agreement', '.']
[]
[[0, 0, 'Material'], [10, 10, 'OtherScientificTerm'], [17, 20, 'OtherScientificTerm']]
 
['The', 'agreement', 'in', 'question', 'involves', 'number', 'in', 'nouns', 'and', 'reflexive', 'pronouns', 'and', 'is', 'syntactic', 'rather', 'than', 'semantic', 'in', 'nature', 'because', 'grammatical', 'number', 'in', 'English', ',', 'like', 'grammatical', 'gender', 'in', 'languages', 'such', 'as', 'French', ',', 'is', 'partly', 'arbitrary', '.']
[[29, 29, 31, 32, 'CONJUNCTION'], [48, 49, 51, 51, 'FEATURE-OF'], [54, 54, 51, 51, 'HYPONYM-OF']]
[[23, 23, 'Generic'], [29, 29, 'OtherScientificTerm'], [31, 32, 'OtherScientificTerm'], [42, 43, 'OtherScientificTerm'], [45, 45, 'Material'], [48, 49, 'OtherScientificTerm'], [51, 51, 'Material'], [54, 54, 'Material']]
 
['The', 'forma

In [46]:
def generate_relations(document):
    relations = list()
    for relation in document.relations:
        arguments = list()
        relation_type = relation['relation_type']
        for argument in relation['arguments']:
            argument_entity = document.entities[argument['entity_id']]
            arguments += [list(range(argument_entity['start'], argument_entity['end']))]
        for old_relation in relations:
            if relation_type == old_relation['type']:
                old_relation['arguments'].append(arguments)
                continue
        relations += [{'type': relation_type, 'arguments': [arguments]}]
    return relations

In [47]:
relations = generate_relations(document)
print(relations)

TypeError: list indices must be integers or slices, not str

In [46]:
def generate_relation(document):
    for relations_in_sentence, sentence_start in zip(document.relations, document.sentence_start):
        relations = list()
        type_set = set()
        for relation in relations_in_sentence:
#             'arguments': [['Arg-1', [9]], ['Arg-2', [14]]]
            arguments = [list(range(relation[0]-sentence_start, relation[1]+1-sentence_start)),
                         list(range(relation[2]-sentence_start,relation[3]+1-sentence_start))]
            relation_type = relation[4].split('.')[0]
            if relation_type in type_set:
                for old_relation in relations:
                    if relation_type == old_relation['type']:
                        old_relation['arguments'].append(arguments)
            else:
                type_set.add(relation_type)
                relations += [{'type': relation_type, 'arguments': [arguments]}]
        print(relations)

In [53]:
generate_relation(document)

AttributeError: 'Event' object has no attribute 'sentence_start'

In [84]:
Name_Entity_Type = set()
for ner, sentence, events_in_sentence, sentence_start in zip(document.ner , document.sentences, document.events, document.sentence_start):
    events = list()
    for event in events_in_sentence:
        trigger, event_type = event[0]
        trigger_ner = ner
        trigger -= sentence_start

        suptype, subtype = event_type.split('.')

        if type_format == 'subtype':
            event_type = subtype
        elif type_format == 'suptype':
            event_type = suptype
        else:
            event_type = suptype + type_format + subtype

        arguments = list()
        for start, end, role in event[1:]:
            start -= sentence_start
            end -= sentence_start
            arguments += [[role, list(range(start, end + 1))]]

        for argument in arguments:
            for ner_pos in ner:
                Name_Entity_Type.add(ner_pos[2])
                if((ner_pos[0]-sentence_start) == argument[1][0]):
                    argument.insert(1, ner_pos[2])
            if(len(argument) != 3):
                print("Wrong")

        event = {'type': event_type, 'tokens': [trigger], 'arguments': arguments}

        events += [event]
#         print(events)
        if(len(event['arguments']) >1):
            A_predict = {'tokens': sentence, 'events': events}
# print(A_predict)
print(Name_Entity_Type)

AttributeError: 'Event' object has no attribute 'ner'

In [85]:
list_A = ['ORG', 'VEH', 'WEA', 'LOC', 'FAC', 'PER', 'GPE']
for a in list_A:
    entity_token = tokenizer.encode(a)
    print(entity_token)
print(tokenizer.encode('<extra_id_0>'))

[4674, 517, 1]
[3, 8575, 566, 1]
[9664, 188, 1]
[301, 5618, 1]
[377, 5173, 1]
[3, 8742, 1]
[350, 5668, 1]
[32099, 1]


In [90]:
entity_dic = {'ORG', 'VEH', 'WEA', 'LOC', 'FAC', 'PER', 'GPE'}
tokenizer.add_special_tokens({"additional_special_tokens": list_A})
model.resize_token_embeddings(len(tokenizer))
print(tokenizer.convert_tokens_to_ids(list_A))
if 'ORG' in list_A:
    print("Wrong")
    
print(tokenizer.decode(32103))
print(tokenizer.pad_token_id)
a = 1.213232
b = round(a, 4)
print(b)

[32100, 32101, 32102, 32103, 32104, 8742, 32105]
Wrong
LOC
0
1.2132


# OneIE data processing

In [29]:
events = list()
# print(document.events)
# print(document.entities)
for event, entity in zip(document.events, document.entities):
#     print(event)
#     print(entity)
    arguments = list()
    for argument in event['arguments']:
        argument_entity = document.entities[argument['entity_id']]
#         print("argument_entity", argument_entity)
        arguments += [[argument['role'], argument_entity['entity_type'] ,list(range(argument_entity['start'], argument_entity['end']))]]

    suptype, subtype = event['event_type'].split(':')

    if type_format == 'subtype':
        event_type = subtype
    elif type_format == 'suptype':
        event_type = suptype
    else:
        event_type = suptype + type_format + subtype

    events += [{
        'type': event_type,
        'tokens': list(range(event['trigger']['start'], event['trigger']['end'])),
        'arguments': arguments
    }]
    print(events)

[{'type': 'Attack', 'tokens': [8], 'arguments': []}]
[{'type': 'Attack', 'tokens': [8], 'arguments': []}, {'type': 'End-Position', 'tokens': [10], 'arguments': [['Person', 'PER', [12, 13]], ['Entity', 'GPE', [17]]]}]


In [ ]:
for ner, sentence, events_in_sentence, sentence_start in zip(document.ner , document.sentences, document.events, document.sentence_start):
        if(len(ner) > 0):
            print((ner))

# Annotated from event text to tree

In [59]:
from data_convert.utils import read_file, check_output, data_counter_to_table, get_schema, output_schema
from nltk.corpus import stopwords

type_start = '<extra_id_0>'
type_end = '<extra_id_1>'
role_start = '<extra_id_2>'
role_end = '<extra_id_3>'

def get_str_from_tokens(tokens, sentence, separator=' '):
    start, end_exclude = tokens[0], tokens[-1] + 1
    return separator.join(sentence[start:end_exclude])

In [43]:
event_schema_set = set()

for event in A_predict['events']:
    print(event)
    event_schema_set = event_schema_set | get_schema(event)
    sep = ' '
    predicate = sep.join([A_predict['tokens'][index]
                          for index in event['tokens']])
#     counter['pred'].update([predicate])
#     counter['type'].update([event['type']])
#     data_counter[in_filename].update(['event'])
#     for argument in event['arguments']:
#         data_counter[in_filename].update(['argument'])
#         counter['role'].update([argument[0]])

print(predicate)
print(event_schema_set)

{'type': 'Attack', 'tokens': [7], 'arguments': [['Attacker', 'PER', [5]], ['Attacker', 'PER', [14]], ['Attacker', 'PER', [19]]]}
fight
{('Attack', 'Attacker')}


In [29]:
tokens=A_predict['tokens']
predicate_arguments=A_predict['events']

token_separator = ' '

event_str_rep_list = list()

for predicate_argument in predicate_arguments:
    event_type = predicate_argument['type']

    # predicate_argument['tokens'] is the trigger index
    # tokens is the sentence tokens, we get the trigger text span here
    predicate_text = get_str_from_tokens(predicate_argument['tokens'], tokens, separator=token_separator)

    # prefix_tokens[predicate_argument['tokens'][0]] = ['[ ']
    # suffix_tokens[predicate_argument['tokens'][-1]] = [' ]']

    role_str_list = list()
    # role_name is the argument role, role_tokens are corresponding text span index
    for role_name, role_entity, role_tokens in predicate_argument['arguments']:
        # if role_name == 'Place' or role_name.startswith('Time'):
        if role_name == event_type:
            continue
        # get the role text span from role tokens index
        role_text = get_str_from_tokens(role_tokens, tokens, separator=token_separator)
#         print(role_text)
#         print(role_entity)
        if False:
            role_str = ' '.join([role_start, role_name, role_entity ,role_text, role_end])
        else:
            role_str = ' '.join([type_start, role_name,role_entity ,role_text, type_end])
        # All arguments in the sentence
#         print(role_str)
        role_str_list += [role_str]
    role_str_list_str = ' '.join(role_str_list)
    event_str_rep = f"{type_start} {event_type} {predicate_text} {role_str_list_str} {type_end}"
    event_str_rep_list += [event_str_rep]

# print(tokens)    
source_text = token_separator.join(tokens)
target_text = ' '.join(event_str_rep_list)

if not False:
    target_text = f'{type_start} ' + \
                    ' '.join(event_str_rep_list) + f' {type_end}'

print(source_text) 
print(" ")
print(target_text)

Britain has deployed some 45,000 troops to fight with the more than 250,000 US soldiers lined up against Iraqi troops .
 
<extra_id_0> <extra_id_0> Attack fight <extra_id_0> Attacker PER troops <extra_id_1> <extra_id_0> Attacker PER soldiers <extra_id_1> <extra_id_0> Attacker PER troops <extra_id_1> <extra_id_1> <extra_id_1>


In [92]:
a = ["%s%s" % ("type_start  ", "type_end")] * 2
print(a)

['type_start  type_end', 'type_start  type_end']


# Get Label Name

In [50]:
def get_label_name_tree(label_name_list, tokenizer, end_symbol='<end>'):
    # Change recurring into non-recurring labels, 
    sub_token_tree = dict()

    # this is label_name token ids
    label_tree = dict()
    for typename in label_name_list:
#         print(typename)
        after_tokenized = tokenizer.encode(typename)
        label_tree[typename] = after_tokenized
    print(label_tree)
    for _, sub_label_seq in label_tree.items():
        # sub_label_seq is the tokenize_ids of typename
        parent = sub_token_tree
        for value in sub_label_seq:
            if value not in parent:
                parent[value] = dict()
            parent = parent[value]
        parent[end_symbol] = None

    return sub_token_tree

In [51]:
from extraction.event_schema import EventSchema
event_schema = './data/text2tree/dyiepp_ace2005_subtype/event.schema'
decoding_type_schema = EventSchema.read_from_file(event_schema)
# print(decoding_type_schema.type_list)
# print(decoding_type_schema.role_list)
# print(decoding_type_schema.type_role_dict)
type_tree = get_label_name_tree(decoding_type_schema.type_list, tokenizer, end_symbol='<tree-end>')

{'Extradite': [8505, 10700, 1], 'Sentence': [4892, 17, 1433, 1], 'Trial-Hearing': [20660, 18, 3845, 9, 1007, 1], 'Die': [316, 1], 'Elect': [3, 21543, 1], 'Declare-Bankruptcy': [28596, 60, 18, 21347, 9433, 75, 63, 1], 'Appeal': [25024, 1], 'Attack': [24655, 1], 'Merge-Org': [4039, 397, 18, 7395, 122, 1], 'Charge-Indict': [15907, 18, 1570, 12194, 1], 'Release-Parole': [13048, 18, 13212, 32, 109, 1], 'Divorce': [2043, 1967, 565, 1], 'Pardon': [2180, 2029, 1], 'Sue': [17564, 1], 'Start-Position': [3273, 18, 345, 32, 7, 4749, 1], 'Meet': [12325, 1], 'Demonstrate': [15782, 29, 7, 17, 2206, 1], 'Execute': [25183, 15, 1], 'Convict': [1193, 7287, 17, 1], 'Transport': [7608, 1], 'Transfer-Ownership': [9900, 18, 667, 210, 687, 2009, 1], 'Arrest-Jail': [1533, 6216, 18, 683, 9, 173, 1], 'Start-Org': [3273, 18, 7395, 122, 1], 'Transfer-Money': [9900, 18, 9168, 15, 63, 1], 'Phone-Write': [8924, 18, 24965, 15, 1], 'End-Position': [3720, 18, 345, 32, 7, 4749, 1], 'Fine': [11456, 1], 'End-Org': [3720, 1

In [75]:
# print(list(type_tree.keys()))
# print(" ")
subtree = type_tree[9900][18][667]
print(len(subtree))
print(subtree)
if '<tree-end>' in subtree:
    print("end_tree")

1
{210: {687: {2009: {1: {'<tree-end>': None}}}}}


In [61]:
role_tree = get_label_name_tree(decoding_type_schema.role_list, tokenizer, end_symbol='<tree-end>')
print(role_tree.keys())
print(role_tree)

{'Plaintiff': [30837, 1], 'Instrument': [13507, 1], 'Person': [5780, 1], 'Entity': [4443, 485, 1], 'Beneficiary': [30570, 1208, 1], 'Adjudicator': [1980, 14312, 447, 1016, 1], 'Seller': [19980, 1], 'Prosecutor': [749, 7, 15, 3044, 127, 1], 'Artifact': [1261, 23, 8717, 1], 'Vehicle': [15095, 1], 'Attacker': [24655, 49, 1], 'Origin': [19477, 1], 'Victim': [12060, 2998, 1], 'Defendant': [3, 16196, 989, 288, 1], 'Agent': [8628, 1], 'Org': [955, 122, 1], 'Buyer': [19099, 1], 'Place': [3399, 1], 'Destination': [19344, 257, 1], 'Target': [12615, 1], 'Recipient': [419, 3389, 4741, 1], 'Giver': [6434, 52, 1]}
dict_keys([30837, 13507, 5780, 4443, 30570, 1980, 19980, 749, 1261, 15095, 24655, 19477, 12060, 3, 8628, 955, 19099, 3399, 19344, 12615, 419, 6434])
{30837: {1: {'<tree-end>': None}}, 13507: {1: {'<tree-end>': None}}, 5780: {1: {'<tree-end>': None}}, 4443: {485: {1: {'<tree-end>': None}}}, 30570: {1208: {1: {'<tree-end>': None}}}, 1980: {14312: {447: {1016: {1: {'<tree-end>': None}}}}}, 19

In [62]:
print(role_tree)
if('<tree-end>' in role_tree):
    print("Wrong")

{30837: {1: {'<tree-end>': None}}, 13507: {1: {'<tree-end>': None}}, 5780: {1: {'<tree-end>': None}}, 4443: {485: {1: {'<tree-end>': None}}}, 30570: {1208: {1: {'<tree-end>': None}}}, 1980: {14312: {447: {1016: {1: {'<tree-end>': None}}}}}, 19980: {1: {'<tree-end>': None}}, 749: {7: {15: {3044: {127: {1: {'<tree-end>': None}}}}}}, 1261: {23: {8717: {1: {'<tree-end>': None}}}}, 15095: {1: {'<tree-end>': None}}, 24655: {49: {1: {'<tree-end>': None}}}, 19477: {1: {'<tree-end>': None}}, 12060: {2998: {1: {'<tree-end>': None}}}, 3: {16196: {989: {288: {1: {'<tree-end>': None}}}}}, 8628: {1: {'<tree-end>': None}}, 955: {122: {1: {'<tree-end>': None}}}, 19099: {1: {'<tree-end>': None}}, 3399: {1: {'<tree-end>': None}}, 19344: {257: {1: {'<tree-end>': None}}}, 12615: {1: {'<tree-end>': None}}, 419: {3389: {4741: {1: {'<tree-end>': None}}}}, 6434: {52: {1: {'<tree-end>': None}}}}


In [77]:
first_tree = role_tree[30837]
print(first_tree[1])
if '<tree-end>' in first_tree[1]:
    print("Wrong")

{'<tree-end>': None}
Wrong


# Transform output to Tree

In [54]:
def convert_bracket(_text):
    # replace the special token labels to Formal statement
    _text = add_space(_text)
    for start in [role_start, type_start]:
        _text = _text.replace(start, left_bracket)
    for end in [role_end, type_end]:
        _text = _text.replace(end, right_bracket)
    return _text

def add_space(text):
    """
    add space between special token
    :param text:
    :return:
    """
    new_text_list = list()
    for item in zip(split_bracket.findall(text), split_bracket.split(text)[1:]):
        new_text_list += item
    return ' '.join(new_text_list)

def get_tree_str(tree):
    """
    get str from event tree
    :param tree:
    :return:
    """
    str_list = list()
    for element in tree:
        if isinstance(element, str):
            str_list += [element]
    return ' '.join(str_list)

In [56]:
import re
left_bracket = '【'
right_bracket = '】'
text = "<extra_id_0> <extra_id_0> ORG-AFF <extra_id_0> Minister <extra_id_0> British <extra_id_1> <extra_id_1> ORG-AFF <extra_id_1> <extra_id_0> PART-WHOLE <extra_id_0> Grand Hotel Europe <extra_id_0> Saint Petersburg <extra_id_1> <extra_id_1> PART-WHOLE <extra_id_1> <extra_id_1>"
from nltk.tree import ParentedTree
brackets = left_bracket + right_bracket
print(text)

<extra_id_0> <extra_id_0> ORG-AFF <extra_id_0> Minister <extra_id_0> British <extra_id_1> <extra_id_1> ORG-AFF <extra_id_1> <extra_id_0> PART-WHOLE <extra_id_0> Grand Hotel Europe <extra_id_0> Saint Petersburg <extra_id_1> <extra_id_1> PART-WHOLE <extra_id_1> <extra_id_1>


In [57]:
split_bracket = re.compile(r"<extra_id_\d>")
new_text_list = list()
for item in zip(split_bracket.findall(text), split_bracket.split(text)[1:]):
    new_text_list += item
print(' '.join(new_text_list))


<extra_id_0>   <extra_id_0>  ORG-AFF  <extra_id_0>  Minister  <extra_id_0>  British  <extra_id_1>   <extra_id_1>  ORG-AFF  <extra_id_1>   <extra_id_0>  PART-WHOLE  <extra_id_0>  Grand Hotel Europe  <extra_id_0>  Saint Petersburg  <extra_id_1>   <extra_id_1>  PART-WHOLE  <extra_id_1>   <extra_id_1> 


In [60]:
new_text = convert_bracket(text)
print(new_text)

【   【  ORG-AFF  【  Minister  【  British  】   】  ORG-AFF  】   【  PART-WHOLE  【  Grand Hotel Europe  【  Saint Petersburg  】   】  PART-WHOLE  】   】 


In [61]:
gold_tree = ParentedTree.fromstring(new_text, brackets=brackets)

In [71]:
print(gold_tree)

(
  (ORG-AFF (Minister (British )) ORG-AFF)
  (PART-WHOLE (Grand Hotel Europe (Saint Petersburg)) PART-WHOLE))


In [78]:
str_list = list()
for relation_tree in gold_tree:
    print(relation_tree.label())
    for role_tree in relation_tree:
        if isinstance(role_tree, str):
            print("end", role_tree)
        else:
            role1_text = role_tree.label() + ' ' + get_tree_str(role_tree)
            role2_text = role_tree[-1].label() + ' '+get_tree_str(role_tree[-1])
            print(role1_text + "  aa  "+ role2_text)
#         print(role_tree.label()+ ' ' +a)
#         print(role_tree[-1].label() + ' '+get_tree_str(role_tree[-1]))
#         print(role_tree[0])

ORG-AFF
Minister   aa  British 
end ORG-AFF
PART-WHOLE
Grand Hotel Europe  aa  Saint Petersburg
end PART-WHOLE


In [148]:
kind = ' '.join(str_list)
print(kind)
new_kind = ' '.join(kind.split(' ')[1:])
print(new_kind)

In [149]:
b = []
a = ["LOC", "ORG", "VEH", "FAC", "PER", "WEA", "GPE"]
a.append(" ")
b =a
print(b)

['LOC', 'ORG', 'VEH', 'FAC', 'PER', 'WEA', 'GPE', ' ']


In [27]:
c = b+[1]
print(c)

['LOC', 'ORG', 'VEH', 'FAC', 'PER', 'WEA', 'GPE', ' ', 1]


In [67]:
print(tokenizer.encode(" "))

[1]


In [94]:
a = tokenizer.decode(3)
b = [1,2,3]
b.append(a)
print(b)

[1, 2, 3, '']


In [100]:
c = [0,1]
print(c+a)

[0, 1, 1548, 3, 6, 38, 17952, 3859, 3292, 3457, 3, 6, 79, 33, 6326, 53, 3, 9, 775, 682, 13, 4719, 3, 6, 652, 5413, 13, 8, 7749, 9534, 45, 8, 10101, 3, 5]


In [95]:
a =  [1548, 3, 6, 38, 17952, 3859, 3292, 3457, 3, 6, 79, 33, 6326, 53, 3, 9, 775, 682, 13, 4719, 3, 6, 652, 5413, 13, 8, 7749, 9534, 45, 8, 10101, 3, 5]
b = list()
for x in a:
    b.append(tokenizer.decode(x))
print(' '.join(b))

Well  , as coalition forces push north  , they are encounter ing  a unique problem of combat  , getting rid of the weapons captured from the enemy  .


In [99]:
import torch
labels = torch.tensor(a)
decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=False)
print(decoded_labels)
print(' '.join(decoded_labels))

['Well', '', ',', 'as', 'coalition', 'forces', 'push', 'north', '', ',', 'they', 'are', 'encounter', 'ing', '', 'a', 'unique', 'problem', 'of', 'combat', '', ',', 'getting', 'rid', 'of', 'the', 'weapons', 'captured', 'from', 'the', 'enemy', '', '.']
Well  , as coalition forces push north  , they are encounter ing  a unique problem of combat  , getting rid of the weapons captured from the enemy  .


In [105]:
print(type([3]))

<class 'list'>


In [217]:
special_token_set = {-1,-2}
tgt_generated = [1,2,3,-1,2,3,2,-1,4,3,-2,4,-2]
special_index_token = list(filter(lambda x: x[1] in special_token_set, list(enumerate(tgt_generated))))
print(special_index_token)

[(3, -1), (7, -1), (10, -2), (12, -2)]


In [218]:
Target = [0, 32099, 32099, 4674, 517, 18, 188, 9089, 32099, 7471, 32099, 13143, 32099, 1]
Target = [0, 32099, 32099, 3, 19846, 18, 518, 6299, 3765, 32099, 3, 19600, 989, 23, 12489, 32099, 783, 563, 32099, 1]
text = tokenizer.decode(Target)
print(text)

<pad><extra_id_0><extra_id_0> PART-WHOLE<extra_id_0> Vivendi Universal<extra_id_0> media group<extra_id_0></s>


In [ ]:
Source = [2, 0, 7, 7, 7, 5]
text2 = tokenizer.decode(Source)
print(text2)

In [225]:
add_special_tokens
print(tokenizer.sep_token)

Using sep_token, but it is not set yet.


None


# Test List in Python

In [82]:
record_list = list()
record = {'relation': int}
record['relation'] = 1
record_list += [record]
print(record_list)

[{'relation': 1}]


In [83]:
record['relation'] = 2
record_list += [record]
print(record_list)

[{'relation': 2}, {'relation': 2}]


In [86]:
tokenizer.decode(2)

'<unk>'

In [7]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

def extract_triplets(text):
    triplets = []
    relation, subject, relation, object_ = '', '', '', ''
    text = text.strip()
    current = 'x'
    for token in text.replace("<s>", "").replace("<pad>", "").replace("</s>", "").split():
        if token == "<triplet>":
            current = 't'
            if relation != '':
                triplets.append({'head': subject.strip(), 'type': relation.strip(),'tail': object_.strip()})
                relation = ''
            subject = ''
        elif token == "<subj>":
            current = 's'
            if relation != '':
                triplets.append({'head': subject.strip(), 'type': relation.strip(),'tail': object_.strip()})
            object_ = ''
        elif token == "<obj>":
            current = 'o'
            relation = ''
        else:
            if current == 't':
                subject += ' ' + token
            elif current == 's':
                object_ += ' ' + token
            elif current == 'o':
                relation += ' ' + token
    if subject != '' and relation != '' and object_ != '':
        triplets.append({'head': subject.strip(), 'type': relation.strip(),'tail': object_.strip()})
    return triplets

# Load model and tokenizer
tokenizer = AutoTokenizer.from_pretrained("Babelscape/rebel-large")
model = AutoModelForSeq2SeqLM.from_pretrained("Babelscape/rebel-large")
gen_kwargs = {
    "max_length": 256,
    "length_penalty": 0,
    "num_beams": 3,
    "num_return_sequences": 3,
}

# Text to extract triplets from
text = 'Punta Cana is a resort town in the municipality of Higüey, in La Altagracia Province, the easternmost province of the Dominican Republic.'

# Tokenizer text
model_inputs = tokenizer(text, max_length=256, padding=True, truncation=True, return_tensors = 'pt')

# Generate
generated_tokens = model.generate(
    model_inputs["input_ids"].to(model.device),
    attention_mask=model_inputs["attention_mask"].to(model.device),
    **gen_kwargs,
)

# Extract text
decoded_preds = tokenizer.batch_decode(generated_tokens, skip_special_tokens=False)

# Extract triplets
for idx, sentence in enumerate(decoded_preds):
    print(f'Prediction triplets sentence {idx}')
    print(extract_triplets(sentence))

Downloading:   0%|          | 0.00/1.51G [00:00<?, ?B/s]

OSError: Unable to load weights from pytorch checkpoint file for 'Babelscape/rebel-large' at '/home/wufeng/.cache/huggingface/transformers/0759d6afb2b4885699bd1f948cbbc28d9da3f97f6c71477ea72027efc50357b5.254c1b93946ad463de37ace02ecc0041c706e27f91f31c68a60ac76c340c9b88'. If you tried to load a PyTorch model from a TF 2.0 checkpoint, please set from_tf=True.

In [6]:
tokenizer = AutoTokenizer.from_pretrained("Babelscape/rebel-large")
model_name_or_path = "Babelscape/rebel-large"
model = AutoModelForSeq2SeqLM.from_pretrained(model_name_or_path)

OSError: Unable to load weights from pytorch checkpoint file for 'Babelscape/rebel-large' at '/home/wufeng/.cache/huggingface/transformers/0759d6afb2b4885699bd1f948cbbc28d9da3f97f6c71477ea72027efc50357b5.254c1b93946ad463de37ace02ecc0041c706e27f91f31c68a60ac76c340c9b88'. If you tried to load a PyTorch model from a TF 2.0 checkpoint, please set from_tf=True.

# Get  the label data

In [27]:
output_filename = './data/new_text2tree/ace_label/'
if not os.path.exists(output_filename):
        os.makedirs(output_filename)
import json
with open('./data/new_text2tree/one_ie_ace2005_subtype/test.json', 'r', encoding="utf-8") as f:
    # 读取所有行 每行会是一个字符串
    for jsonstr in f.readlines():
        # 将josn字符串转化为dict字典
        jsonstr = json.loads(jsonstr)
        with open(output_filename + 'test_end_before.json', 'a+', encoding='utf-8') as f2:
            line = json.dumps(jsonstr["relation"], ensure_ascii=False)
            f2.write(line+'\n')
